Дисциплина: Advnced Machine Learning<br>
Домашнее задание №3<br>
Студент: Акопян Артем<br>
Группа: DS-22

# Homework 3

In [24]:
from typing import Tuple
from collections import Counter
import itertools
import math
import random
import copy
import re

from tqdm import tqdm

import numpy as np
from nltk import everygrams

import matplotlib.pyplot as plt

In [25]:
NOVELS = ["Karenina", "WaP", "WaP_en"]
EXAMPLES = [
"""Минуты через две из двери бодрым шагом вышла, быстро повернулась и стала подле \
надзирателя невысокая и очень полногрудая молодая женщина в сером халате, надетом \
на белую кофту и на белую юбку. На ногах женщины были полотняные чулки, на \
чулках — острожные коты, голова была повязана белой косынкой, из-под которой, очевидно \
умышленно, были выпущены колечки вьющихся черных волос. Все лицо женщины было той \
особенной белизны, которая бывает на лицах людей, проведших долгое время взаперти, и \
которая напоминает ростки картофеля в подвале.""",
            
"""What did it matter where you lay once you were dead? \
In a dirty sump or in a marble tower on top of a high hill? \
You were dead, you were sleeping the big sleep, you were not bothered by things like that."""
]

SPACE = [" "]
ALFA_RUS = "а б в г д е ё ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я".split() + SPACE
ALFA_ENG = [chr(ord("a") + i) for i in range(26)] + SPACE

In [26]:
RUS_RE = re.compile("[^а-я ]*")
ENG_RE = re.compile("[^a-z ]*")

In [27]:
# Load corpora
texts = []

with open("AnnaKarenina.txt", "r", encoding="UTF-8") as handler:
    text = handler.read()
    texts.append(re.sub(RUS_RE, "", text.lower()))
    
with open("WarAndPeace.txt", "r", encoding="UTF-8") as handler:
    text = handler.read()
    texts.append(re.sub(RUS_RE, "", text.lower()))
    
with open("WarAndPeaceEng.txt", "r", encoding="UTF-8") as handler:
    text = handler.read()
    texts.append(re.sub(ENG_RE, "", text.lower()))

In [28]:
# For further use let's preprocess the example text in Russian
fragment = re.sub(RUS_RE, "", EXAMPLES[0].lower())
fragment

'минуты через две из двери бодрым шагом вышла быстро повернулась и стала подле надзирателя невысокая и очень полногрудая молодая женщина в сером халате надетом на белую кофту и на белую юбку на ногах женщины были полотняные чулки на чулках  острожные коты голова была повязана белой косынкой изпод которой очевидно умышленно были выпущены колечки вьющихся черных волос все лицо женщины было той особенной белизны которая бывает на лицах людей проведших долгое время взаперти и которая напоминает ростки картофеля в подвале'

## Part 1
### Реализуйте базовый частотный метод по Шерлоку Холмсу
- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе совсем вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом

In [29]:
def freq_dict(
    text: str,
    n_gram: int=1
) -> Counter:
    
    d = Counter()
    for group_begin in range(len(text) + 1 - n_gram):
        group = text[group_begin : group_begin + n_gram]
        d[group] += 1
    for key in d:
        d[key] /= len(text)
    return d


def forward_mapping(
    freqs: dict
) -> dict:
    
    orig = list(freqs.keys())
    subs = np.random.choice(orig, replace=False, size=len(freqs))
    mapping = dict()
    for orig_char, subs_char in zip(orig, subs):
        mapping[orig_char] = subs_char
    return mapping


def reverse_mapping(
    freqs: dict,
    sample_freqs: dict
) -> dict:
    
    freqs_sorted = sorted(freqs.items(), key=lambda x: x[1], reverse=True)
    sample_freqs_sorted = sorted(sample_freqs.items(), key=lambda x: x[1], reverse=True)
    
    reverse_mapping = dict()
    for sample_char, sample_freq in sample_freqs_sorted:
        min_diff = 1
        best_char = None
        for corpus_char, freq in freqs_sorted:
            diff = abs(freq - sample_freq)
            if diff < min_diff:
                best_char = corpus_char
                min_diff = diff

        reverse_mapping[sample_char] = best_char
        freqs_sorted = [(char, freq) for char, freq in freqs_sorted if char !=best_char] 
    return reverse_mapping


def apply_mapping(
    text: str,
    mapping: dict
) -> str:
    
    new_str = ""
    step = len(list(mapping.keys())[0])
    for i in range(0, (len(text) // step * step), step):
        new_str += mapping.get(text[i: i + step])
    return new_str


def decrypting_accuracy(
    text1: str,
    text2: str
) -> float:

    correct = 0
    for char1, char2 in zip(text1, text2):
        correct += int(char1 == char2)
    return correct / len(text1)


def mapping_accuracy(
    mapping1: dict,
    mapping2: list
) -> float:
    
    correct = 0
    for (key1, value1), (key2, value2) in zip(mapping1.items(), mapping2.items()):
        correct += 1 * (mapping2.get(mapping1[key1]) == value2)
    return correct / len(mapping)

In [30]:
corpus_freqs = freq_dict(texts[1])

mapping = forward_mapping(corpus_freqs)
print(mapping)

encoded_text = apply_mapping(fragment, mapping)

{'в': 'п', 'о': 'б', 'й': 'ц', 'н': 'р', 'а': 'м', ' ': 'т', 'и': 'э', 'м': 'о', 'р': 'в', 'с': 'и', 'ы': 'з', 'з': 'ь', 'е': 'д', 'т': 'ж', 'л': 'ч', 'ь': 'к', 'к': 'н', 'ч': 'ы', 'г': 'ф', 'д': 'у', 'у': 'г', 'п': 'е', 'я': 'с', 'ж': 'ъ', 'б': ' ', 'щ': 'а', 'ф': 'ш', 'э': 'щ', 'х': 'я', 'ю': 'х', 'ш': 'й', 'ц': 'ю', 'ъ': 'л'}


In [31]:
fragment_freqs = freq_dict(fragment)
encoded_freqs = freq_dict(encoded_text)
fragment_freqs, encoded_freqs

(Counter({'м': 0.01727447216890595,
          'и': 0.04798464491362764,
          'н': 0.06525911708253358,
          'у': 0.02111324376199616,
          'т': 0.036468330134357005,
          'ы': 0.04030710172744722,
          ' ': 0.15163147792706333,
          'ч': 0.013435700575815739,
          'е': 0.0710172744721689,
          'р': 0.0345489443378119,
          'з': 0.013435700575815739,
          'д': 0.026871401151631478,
          'в': 0.036468330134357005,
          'б': 0.02495201535508637,
          'о': 0.1017274472168906,
          'ш': 0.007677543186180422,
          'а': 0.06525911708253358,
          'г': 0.009596928982725527,
          'л': 0.05566218809980806,
          'с': 0.023032629558541268,
          'п': 0.02111324376199616,
          'ь': 0.005758157389635317,
          'я': 0.02111324376199616,
          'к': 0.028790786948176585,
          'ж': 0.007677543186180422,
          'щ': 0.009596928982725527,
          'х': 0.013435700575815739,
          'ю': 0.0

In [32]:
reverse_unigram_mapping = reverse_mapping(corpus_freqs, encoded_freqs)
print(reverse_unigram_mapping)
decoded_text = apply_mapping(encoded_text, reverse_unigram_mapping)
decoded_text

{'т': ' ', 'б': 'о', 'д': 'а', 'р': 'е', 'м': 'и', 'ч': 'н', 'э': 'т', 'з': 'л', 'ж': 'р', 'п': 'в', 'в': 'к', 'н': 'д', 'у': 'м', ' ': 'у', 'и': 'п', 'г': 'я', 'е': 'г', 'с': 'ь', 'о': 'ы', 'ы': 'б', 'ь': 'з', 'я': 'ч', 'ц': 'й', 'ф': 'ж', 'а': 'ш', 'х': 'х', 'й': 'ю', 'ъ': 'ц', 'к': 'э', 'ш': 'щ', 'ю': 'ф'}


'ытеярл баказ мва тз мвакт уомклы юижоы влюни улпрко говакеянипэ т прини гомна еимзткирань еавлподиь т обаеэ гонеожкямиь ыономиь цаештеи в пакоы чинира еимароы еи уанях дощря т еи уанях худя еи еожич цаештел улнт гонореьела бяндт еи бяндич  опркоцела дорл жонови улни говьзиеи уаной допледой тзгом дорокой обавтмео яылюнаеео улнт влгяшаел донабдт вэхштчпь бакелч воноп впа нтфо цаештел улно рой опоуаееой уантзел дорокиь улвиар еи нтфич нхмай гковамютч монжоа вкаыь взигакрт т дорокиь еигоытеиар копрдт дикрощань в гомвина'

In [33]:
encoded_text

'оэргжзтыдвдьтупдтэьтупдвэт бувзотймфботпзйчмт зижвбтебпдвргчмиктэтижмчмтебучдтрмуьэвмждчстрдпзибнмстэтбыдрктебчрбфвгумстобчбумстъдраэрмтптидвботямчмждтрмуджботрмт дчгхтнбшжгтэтрмт дчгхтх нгтрмтрбфмятъдраэрзт зчэтебчбжрсрздтыгчнэтрмтыгчнмяттбижвбърздтнбжзтфбчбпмт зчмтебпсьмрмт дчбцтнбизрнбцтэьебутнбжбвбцтбыдпэурбтгозйчдррбт зчэтпзегадрзтнбчдынэтпкхаэяистыдврзятпбчбитпидтчэюбтъдраэрзт зчбтжбцтбиб дррбцт дчэьрзтнбжбвмст зпмджтрмтчэюмятчхудцтевбпдуйэятубчфбдтпвдостпьмедвжэтэтнбжбвмстрмебоэрмджтвбижнэтнмвжбшдчстптебупмчд'

In [34]:
decrypting_accuracy(fragment, decoded_text), mapping_accuracy(mapping, reverse_unigram_mapping)

(0.31477927063339733, 0.12121212121212122)

### Not so bad at all. Let's try to improve it!

## Part 2
Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.

In [35]:
corpus_freqs_bigram = freq_dict(texts[1], n_gram=2)
encoded_freqs_bigram = freq_dict(encoded_text, n_gram=2)

corpus_freqs_sorted = sorted(corpus_freqs_bigram.items(), key=lambda x: x[1], reverse=True)
encoded_freqs_sorted = sorted(encoded_freqs_bigram.items(), key=lambda x: x[1], reverse=True)

In [36]:
encoded_text

'оэргжзтыдвдьтупдтэьтупдвэт бувзотймфботпзйчмт зижвбтебпдвргчмиктэтижмчмтебучдтрмуьэвмждчстрдпзибнмстэтбыдрктебчрбфвгумстобчбумстъдраэрмтптидвботямчмждтрмуджботрмт дчгхтнбшжгтэтрмт дчгхтх нгтрмтрбфмятъдраэрзт зчэтебчбжрсрздтыгчнэтрмтыгчнмяттбижвбърздтнбжзтфбчбпмт зчмтебпсьмрмт дчбцтнбизрнбцтэьебутнбжбвбцтбыдпэурбтгозйчдррбт зчэтпзегадрзтнбчдынэтпкхаэяистыдврзятпбчбитпидтчэюбтъдраэрзт зчбтжбцтбиб дррбцт дчэьрзтнбжбвмст зпмджтрмтчэюмятчхудцтевбпдуйэятубчфбдтпвдостпьмедвжэтэтнбжбвмстрмебоэрмджтвбижнэтнмвжбшдчстптебупмчд'

In [37]:
reverse_bigram_mapping = reverse_mapping(corpus_freqs_bigram, encoded_freqs_bigram)
print(reverse_bigram_mapping)
decoded_text = apply_mapping(encoded_text, reverse_bigram_mapping)
decoded_text

{'эт': 'о ', 'т ': 'и ', 'мт': 'а ', 'рм': 'е ', 'тр': ' с', 'тп': ' п', 'ст': ' в', 'еб': ' н', 'тн': '  ', 'нб': 'то', 'дт': ' о', 'те': 'я ', 'др': ' к', 'бч': ' и', 'рз': 'ст', 'зт': 'на', 'дв': 'ь ', 'тэ': 'го', ' з': 'ал', 'вб': 'не', 'дч': 'по', 'чб': 'но', 'жб': 'ра', 'цт': 'ко', 'эр': 'ов', 'бу': 'л ', 'чм': 'й ', 'бт': 'ка', 'мс': ' б', 'рб': 'м ', ' д': 'во', 'ят': 'ро', 'чэ': ' д', 'бж': 'ер', 'би': 'ол', 'бц': 'ен', 'ты': ' т', 'ыд': 'он', 'пд': 'от', 'от': 'ни', 'бо': 'ос', 'иж': 'ла', 'бп': 'ли', 'чд': 'у ', 'тб': 'ло', 'аэ': 'ор', 'зч': 'в ', 'ту': 'ва', 'уп': 'пр', 'эь': 'ел', 'фб': 'ре', 'пз': ' м', 'гч': 'за', 'мч': 'та', 'вм': 'ри', 'нм': 'ан', 'тъ': ' ч', 'ъд': ' г', 'ра': 'ил', 'дж': 'ть', 'мя': 'к ', 'нэ': 'ом', 'пм': 'ат', 'бв': 'ск', 'тч': 'ог', 'оэ': ' е', 'рг': 'аз', 'жз': 'да', 'вд': ' у', 'ьт': 'ас', 'зй': 'ем', 'йч': ' з', 'зи': 'же', 'жв': 'ле', 'вр': 'с ', 'кт': 'ве', 'ти': 'ой', 'му': 'ет', 'мж': 'бы', 'жд': 'ы ', 'чс': 'мо', 'дп': ' р', 'иб': 'де', 'бы

' еазда ть беваотгоаспрь о дроечаадэтос пемй и желека нотс зашевео латаа  нля ое муылбыпо вик мдеан во до кве нспми жав вхоноав в гилова акльненикнй ы  сеттьос са вочттитоглняо е и пон ейожняе  смик  ч корсти в о  нноиеязод тзаом са итськ куоллезнст отодаул илиа алй я липеыма вонокото эсоенгоутл   ерскенлооницсакаусему викаал д пшалс кнатоу жао оч лю уж ть строр новшуг о двы ч корсти в каракоолсе км ково дамнаторари валатть са  дбрроажесколулииччиропабуба п унд пт з обо о торари ве  н ее тьаяолопо анобегпо вак нпрта'

In [38]:
fragment

'минуты через две из двери бодрым шагом вышла быстро повернулась и стала подле надзирателя невысокая и очень полногрудая молодая женщина в сером халате надетом на белую кофту и на белую юбку на ногах женщины были полотняные чулки на чулках  острожные коты голова была повязана белой косынкой изпод которой очевидно умышленно были выпущены колечки вьющихся черных волос все лицо женщины было той особенной белизны которая бывает на лицах людей проведших долгое время взаперти и которая напоминает ростки картофеля в подвале'

In [39]:
decrypting_accuracy(fragment, decoded_text)

0.11516314779270634

### It has gotten even worse. Nice job!

## Part 3
Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:

- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.


In [47]:
def smooth_freq(
    text: str
) -> dict:

    freqs = dict()
    vocab_len = len(set(text)) ** 2
    text = ["".join(bigram) for bigram in everygrams(text, min_len=2, max_len=2)]
    for k, v in Counter(text).items():
        freqs[k] = (v + 1) / (len(text) + len(set(text)) ** 2)
    return freqs


def text_proba(
    text: str,
    mapping: dict,
    freqs: dict
) -> np.array:

    decoded_text = apply_mapping(text, mapping)
    log_proba = 0
    for i in range(len(decoded_text) - 2):
        bigram = decoded_text[i: i + 2]
        bigram_proba = freqs.get(bigram)
        if bigram_proba is None:
            bigram_proba = 1 / (len(text) + len(ALFA_RUS) ** 2)
        log_proba += np.log(bigram_proba)
    return log_proba
     
    
def reverse_mapping_mcmc(
    encoded_text: str,
    abc_encoded: list,
    abc_corpus: list,
    freqs_corpus: dict
) -> Tuple:

    count = 0
    best_mapping = None
    all_mappings = []
    best_log_likekihood = -np.inf
    for trial in tqdm(range(10), leave=False, position=0, total=10):
        abc_iter = abc_corpus
        reverse_mapping = {k: v for k, v in zip(abc_encoded, abc_iter[:len(abc_encoded)])}
        current = text_proba(encoded_text, reverse_mapping, freqs_corpus)
        for i in range(21000):
            abc_proposal = abc_iter[:]
            x, y = np.random.choice(len(abc_proposal), replace=False, size=2)
            abc_proposal[x], abc_proposal[y] = abc_proposal[y], abc_proposal[x]
            reverse_mapping_proposal = {k: v for k, v in zip(abc_encoded, abc_proposal[:len(abc_encoded)])}
            proposal = text_proba(encoded_text, reverse_mapping_proposal, freqs_corpus)
            p = np.exp(proposal - current)
            if p > np.random.rand():
                count += 1
                abc_iter = abc_proposal
                current = proposal
                reverse_mapping = reverse_mapping_proposal
        if current > best_log_likekihood:
            best_log_likekihood = current
            best_mapping = reverse_mapping
        all_mappings.append(reverse_mapping)

    print(f"Likelihood: {best_log_likekihood}")
    return best_mapping, all_mappings

In [48]:
freqs_corpus = smooth_freq(texts[0])

In [49]:
best_reverse_mapping, _ = reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=ALFA_RUS, 
    abc_corpus=ALFA_RUS,
    freqs_corpus=freqs_corpus
)

Likelihood: -3082.2423673386534


In [50]:
apply_mapping(encoded_text, best_reverse_mapping)

'минуты череё две иё двери бодрым шагом вышла быстро повернулась и стала подле надёирателя невысокая и очень полногрудая молодая женъина в сером залате надетом на белую кощту и на белую юбку на ногаз женъины были полотняные чулки на чулказ  острожные коты голова была повяёана белой косынкой иёпод которой очевидно умышленно были выпуъены колечки вьюъизся черныз волос все лихо женъины было той особенной белиёны которая бывает на лихаз людей проведшиз долгое время вёаперти и которая напоминает ростки картощеля в подвале'

### Almost!!!

## Part 4
Расшифруйте сообщение:<br>
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ

In [51]:
vsm = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

In [52]:
best_reverse_mapping, _ = reverse_mapping_mcmc(
    vsm, 
    abc_encoded=list(set(vsm)), 
    abc_corpus=ALFA_RUS,
    freqs_corpus=freqs_corpus
)

Likelihood: -1309.8584046230385


In [53]:
apply_mapping(vsm, best_reverse_mapping)

'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаё'

#### Although the vsm (very secret message) sounds as it is read by someone who caught the flu, it is still clear, I hope. The letters "м" & "д" and some other pairs are still not on places.